In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [24]:
model_name = "Qwen/Qwen2.5-1.5B-Instruct"

In [25]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [26]:
with open('ENG_article.txt', 'r', encoding='utf-8') as file:
        article_text = file.read()
        print(f"Файл прочитан. Длина: {len(article_text)} символов")

Файл прочитан. Длина: 8291 символов


In [27]:
#Промпт
system_prompt = """Ты — помощник, анализирующий текст. Прочитай предоставленный текст и на основе только его информации дай точные ответы на вопросы.
Если в тексте нет ответа, честно скажи об этом. Ответы давай на русском языке, цитируй фразы из текста для доказательства."""
system_prompt

'Ты — помощник, анализирующий текст. Прочитай предоставленный текст и на основе только его информации дай точные ответы на вопросы.\nЕсли в тексте нет ответа, честно скажи об этом. Ответы давай на русском языке, цитируй фразы из текста для доказательства.'

In [28]:
#Вопросы
user_questions = """
Ответь на следующие вопросы:

1. В каком году была обозначена проблема взрывающихся градиентов?
2. Кто в 1891 году разработал метод уничтожающей производной?
3. Кто предложил цепное правило дифференцирования и в каком году?
"""


In [29]:
messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": f"Текст статьи:\n{article_text}\n\n{user_questions}"}
]

In [30]:
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [31]:
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

In [32]:
generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512,
    temperature=0.1,
    do_sample=True
)

In [33]:
generated_ids_ = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]
response = tokenizer.batch_decode(generated_ids_, skip_special_tokens=True)[0]

In [34]:
# Сохранение промпта
with open('prompt.txt', 'w', encoding='utf-8') as f:
    f.write(text)
print("Промпт сохранен в файл 'prompt.txt'")

# Сохранение ответов модели
with open('res.txt', 'w', encoding='utf-8') as f:
    f.write(response)
print("Ответы LLM сохранены в файл 'res.txt'")

Промпт сохранен в файл 'prompt.txt'
Ответы LLM сохранены в файл 'res.txt'
